# Sales & Customer Analytics using SQL and Python

### This notebook explores key business metrics including revenue, customer concentration, product performance, churn risk, and purchasing behaviour using SQLite and pandas.

In [56]:
import sqlite3
import pandas as pd
import os

In [57]:
os.makedirs("data", exist_ok=True)

In [58]:
conn = sqlite3.connect("/Users/joshuauvie/Documents/sales-analytics-python-sql/data/sales.db")

# 1. Core transactions table

In [59]:
query = """
SELECT *
FROM transactions;
"""

### This query loads the full transactions dataset to understand the structure, available fields, and overall data quality before analysis.

In [60]:
df = pd.read_sql(query, conn)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
397879,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
397880,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
397881,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
397882,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


# 2. What is the most revenue generated by customers?

In [61]:
query = """
SELECT
    CustomerID,
    SUM(Quantity * UnitPrice) AS total_spent
FROM transactions
GROUP BY CustomerID;
"""

### This query calculates total spending per customer to identify overall revenue contribution and enable customer-level analysis.

In [62]:
customer_revenue = pd.read_sql(query, conn)
customer_revenue.shape

(4338, 2)

In [63]:
customer_revenue

,CustomerID,total_spent
0,12346.0,77183.60
1,12347.0,4310.00
2,12348.0,1797.24
3,12349.0,1757.55
4,12350.0,334.40
...,...,...
4333,18280.0,180.60
4334,18281.0,80.82
4335,18282.0,178.05
4336,18283.0,2094.88


# 3. Who are the Top 10 customers?

In [64]:
query = """
SELECT
    CustomerID,
    ROUND(SUM(Quantity * UnitPrice), 1) AS total_spent
FROM transactions
WHERE Quantity > 0
GROUP BY CustomerID
ORDER BY total_spent DESC
LIMIT 10;
"""

### This query identifies the top 10 customers by total spend, highlighting high-value customers who contribute most to revenue.

In [65]:
top_10_customers = pd.read_sql(query, conn)
top_10_customers

,CustomerID,total_spent
0,14646.0,280206.0
1,18102.0,259657.3
2,17450.0,194550.8
3,16446.0,168472.5
4,14911.0,143825.1
5,12415.0,124914.5
6,14156.0,117379.6
7,17511.0,91062.4
8,16029.0,81024.8
9,12346.0,77183.6


# 4. Which products generate the most revenue?

In [66]:
query = """
SELECT 
    StockCode,
    Description,
    ROUND(SUM(Quantity * UnitPrice), 1) AS revenue
FROM transactions
WHERE Quantity > 0
GROUP BY StockCode, Description
ORDER BY revenue DESC
LIMIT 10;
"""

### This query ranks products by total revenue to determine which items drive the highest sales performance.

In [67]:
product_revenue = pd.read_sql(query, conn)
product_revenue

,StockCode,Description,revenue
0,23843,"PAPER CRAFT , LITTLE BIRDIE",168469.6
1,22423,REGENCY CAKESTAND 3 TIER,142592.9
2,85123A,WHITE HANGING HEART T-LIGHT HOLDER,100448.2
3,85099B,JUMBO BAG RED RETROSPOT,85220.8
4,23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.7
5,POST,POSTAGE,77804.0
6,47566,PARTY BUNTING,68844.3
7,84879,ASSORTED COLOUR BIRD ORNAMENT,56580.3
8,M,Manual,53779.9
9,23084,RABBIT NIGHT LIGHT,51346.2


# 5. What is the monthly revenue trend?

In [68]:
query = """
    SELECT 
        strftime('%Y-%m', InvoiceDate) AS month,
        ROUND(SUM(Quantity * UnitPrice), 1) AS monthly_revenue
    FROM transactions
    WHERE Quantity > 0
    GROUP BY month
"""

### This query aggregates revenue by month to analyse sales trends and identify periods of high or low performance over time.

In [69]:
monthly_revenue = pd.read_sql(query, conn)
monthly_revenue

,month,monthly_revenue
0,2010-12,572713.9
1,2011-01,569445.0
2,2011-02,447137.3
3,2011-03,595500.8
4,2011-04,469200.4
5,2011-05,678594.6
6,2011-06,661213.7
7,2011-07,600091.0
8,2011-08,645343.9
9,2011-09,952838.4


# 6. What percentage of customers are repeat buyers?

In [70]:
query = """
WITH customers_orders AS (
    SELECT 
        CustomerID,
        COUNT(DISTINCT InvoiceNo) AS num_orders
    FROM transactions
    GROUP BY CustomerID
)
    SELECT 
        c.CustomerID,
        c.num_orders,
        ROUND(
            (c.num_orders * 100.0) / 
            (SELECT SUM(num_orders) FROM customers_orders),
            1
        ) AS percent
    FROM customers_orders c 
    ORDER BY percent DESC
    LIMIT 10;
"""

### This query measures customer purchase frequency and calculates each customer’s share of total orders to identify repeat buyers.

In [71]:
freq_customers = pd.read_sql(query, conn)
freq_customers

,CustomerID,num_orders,percent
0,12748.0,209,1.1
1,14911.0,201,1.1
2,17841.0,124,0.7
3,12971.0,86,0.5
4,13089.0,97,0.5
5,14606.0,93,0.5
6,15311.0,91,0.5
7,14646.0,73,0.4
8,13408.0,62,0.3
9,13694.0,50,0.3


# 7. What is the average order value (AOV)?

In [72]:
query = """
WITH order_totals AS (
    SELECT 
        InvoiceNo,
        SUM(Quantity * UnitPrice) AS revenue
    FROM transactions
    WHERE Quantity > 0
    GROUP BY InvoiceNo
)
SELECT
    ROUND(AVG(revenue), 2) AS avg_order_value
FROM order_totals;
"""

### This query computes the average order value by first aggregating revenue at invoice level to avoid item-level bias.

In [73]:
aov = pd.read_sql(query, conn)
aov

,avg_order_value
0,480.87


# 8. Which countries generate the most revenue?

In [74]:
query = """
SELECT
    Country,
    ROUND(SUM(Quantity * UnitPrice), 1) AS revenue
FROM transactions
WHERE Quantity > 0
GROUP BY Country
ORDER BY revenue DESC;
"""

### This query analyses revenue by country to understand geographic performance and identify key international markets.

In [75]:
country_revenue = pd.read_sql(query, conn)
country_revenue

,Country,revenue
0,United Kingdom,7308391.6
1,Netherlands,285446.3
2,EIRE,265545.9
3,Germany,228867.1
4,France,209024.0
5,Australia,138521.3
6,Spain,61577.1
7,Switzerland,56443.9
8,Belgium,41196.3
9,Sweden,38378.3


# 9. What time of day has the highest sales volume?

In [76]:
query = """
SELECT 
    strftime('%H', InvoiceDate) AS hour,
    SUM(Quantity) AS sales_volume
FROM transactions
WHERE Quantity > 0
GROUP BY hour
ORDER BY sales_volume DESC;
"""

### This query examines sales volume by hour of the day to identify peak purchasing times and customer behaviour patterns.

In [77]:
highest_sales = pd.read_sql(query, conn)
highest_sales

,hour,sales_volume
0,12,825218
1,10,775925
2,13,682809
3,11,648457
4,14,557981
5,15,540571
6,09,482329
7,16,254032
8,08,158263
9,17,129577


# 10. Who are customers at risk of churn?

In [78]:
query = """
SELECT 
    CustomerID,
    MAX(InvoiceDate)
FROM transactions
WHERE Quantity > 0
GROUP BY CustomerID
ORDER BY InvoiceDate ASC;
"""

### This query identifies customers with the oldest most recent purchase dates, highlighting customers who may be at risk of churn.

In [79]:
likely_churn = pd.read_sql(query, conn)
likely_churn

,CustomerID,MAX(InvoiceDate)
0,18074.0,2010-12-01 09:53:00
1,13747.0,2010-12-01 10:37:00
2,12791.0,2010-12-01 11:27:00
3,17908.0,2010-12-01 11:45:00
4,16583.0,2010-12-01 12:03:00
...,...,...
4333,17581.0,2011-12-09 12:21:00
4334,13777.0,2011-12-09 12:25:00
4335,15804.0,2011-12-09 12:31:00
4336,13113.0,2011-12-09 12:49:00


# 11. What is the revenue contribution of the top 20% of customers?

In [80]:
query = """
WITH customer_revenue AS (
SELECT
    CustomerID,
    ROUND(SUM(Quantity * UnitPrice), 1) AS revenue
FROM transactions
WHERE Quantity > 0
GROUP BY CustomerID
), 
ranked_customers AS (
SELECT *,
    NTILE(5) OVER (ORDER BY revenue DESC) AS revenue_bucket
    FROM customer_revenue
)
SELECT 
    SUM(revenue) AS top_20_percent_revenue
FROM ranked_customers
WHERE revenue_bucket = 1;
"""

### This query segments customers into revenue quintiles and calculates how much revenue is generated by the top 20% of customers.

In [81]:
rev_contributions_20 = pd.read_sql(query, conn)
rev_contributions_20

,top_20_percent_revenue
0,6649437.2


# Key Insights & Conclusions

- Revenue is highly concentrated, with a small proportion of customers contributing a disproportionately large share of total sales, highlighting the importance of retaining high-value customers.
- A clear group of repeat buyers exists, indicating opportunities for targeted loyalty or retention strategies.
- The top-selling products account for a significant portion of revenue, suggesting that inventory and marketing efforts should prioritise these items.
- Sales exhibit noticeable monthly variation, indicating seasonality effects that could inform forecasting and planning.
- Average Order Value (AOV) provides a stable benchmark for customer spend per transaction and can be used to monitor performance over time.
- Sales volume peaks at specific times of day, offering insight into optimal timing for promotions or operational staffing.
- Revenue contribution analysis shows that the top 20% of customers generate a substantial share of total revenue, consistent with Pareto-style dynamics.

Overall, this analysis demonstrates how SQL and Python can be combined to extract actionable business insights from transactional data, supporting data-driven decision-making across sales, marketing, and customer strategy.
